In [2]:
import os
%cd ..
os.getcwd()

/Users/macbook/Documents/Code/game


'/Users/macbook/Documents/Code/game'

In [3]:
import pygame
from src.animation import Animation
from src.utils import load_dict_from_folder

pygame 2.6.1 (SDL 2.28.4, Python 3.8.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
pygame.init()
WIDTH, HEIGHT = 1280, 720
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
running = True
player_x, player_y = WIDTH // 2, HEIGHT // 2
player_size = 50 # 50 x 50
image_path = os.path.join("images", "backgrounds", "volcano.png")

background = pygame.image.load(image_path)
background = pygame.transform.scale(background, (WIDTH, HEIGHT))


In [5]:
player_dict = load_dict_from_folder("images/characters/mark")

In [12]:
import pygame

class Player:
    def __init__(self, x, y, speed):
        self.x = x
        self.y = y
        self.base_speed = speed
        self.speed = speed  # Maintain current speed separately
        self.state = "idle"
        self.facing_right = True
        self.is_running = False
        self.last_press_d = 0
        self.last_press_a = 0
        
        self.animations = {
            "idle": Animation(player_dict["idle"], 100),
            "move": Animation(player_dict["move"], 100),
            "attack": Animation(player_dict["attack"], 150),
            "run": Animation(player_dict["run"], 150),
        }

    def handle_event(self, event):
        """Handle key presses for running and attacking"""
        current_time = pygame.time.get_ticks()

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_d:
                if current_time - self.last_press_d < 300:  # Double-tap within 300ms
                    self.start_running(direction="right")
                else:
                    self.is_running = False
                self.last_press_d = current_time  # Update last press time
            
            elif event.key == pygame.K_a:
                if current_time - self.last_press_a < 300:  # Double-tap within 300ms
                    self.start_running(direction="left")
                else:
                    self.is_running = False
                self.last_press_a = current_time  # Update last press time
            
            # elif event.key == pygame.K_j:
            #     self.attack()  # Attack when pressing "J"

    def start_running(self, direction):
        self.is_running = True
        self.state = "run"
        self.speed = self.base_speed * 3
        self.facing_right = (direction == "right")
        
    def move(self, keys):
        """Handles movement when keys are held"""
        moving = False
        #handle stopping if player is running
        if self.is_running:
            if (self.facing_right and keys[pygame.K_a]) or (not self.facing_right and keys[pygame.K_d]):
                self.is_running = False
                self.speed = self.base_speed
                self.state = "idle"
            
            elif self.facing_right:
                if self.x + 80 + self.speed < WIDTH:
                    self.x += self.speed  # Move forward
                    moving = True
            else:
                if self.x - self.speed > 0:
                    self.x -= self.speed  # Move backward
                    moving = True
            
            if keys[pygame.K_w]:
                if self.y - self.base_speed > 0:
                    self.y -= self.base_speed
                moving = True

            if keys[pygame.K_s]:
                if self.y + 80 + self.base_speed < HEIGHT:
                    self.y += self.base_speed
                moving = True

                
        if not self.is_running:  # Only move if not running
            if keys[pygame.K_d]:
                self.facing_right = True
                if self.x + 80 + self.base_speed < WIDTH:
                    self.x += self.base_speed
                moving = True

            if keys[pygame.K_a]:
                self.facing_right = False
                if self.x - self.base_speed > 0:
                    self.x -= self.base_speed
                moving = True

            if keys[pygame.K_w]:
                if self.y - self.base_speed > 0:
                    self.y -= self.base_speed
                moving = True

            if keys[pygame.K_s]:
                if self.y + 80 + self.base_speed < HEIGHT:
                    self.y += self.base_speed
                moving = True
        
            if moving:
                self.state = "move"
        if moving == False:
            self.state = "idle"  # No movement, go idle

    def idle(self):
        """Reset player to idle state"""
        self.state = "idle"
        self.speed = self.base_speed

    def attack(self):
        """Player attacks"""
        self.state = "attack"

    def draw(self, screen):
        """Draw the player on screen"""
        frame = self.animations[self.state].get_current_frame()
        if not self.facing_right:
            frame = pygame.transform.flip(frame, True, False)
        screen.blit(frame, (self.x, self.y))


In [ ]:
player = Player(100, 100, speed=2)

running = True
clock = pygame.time.Clock()

while running:
    screen.blit(background, (0, 0))
    keys = pygame.key.get_pressed()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            player.handle_event(event)

    player.move(keys)
    if keys[pygame.K_j]:
        player.attack()
    
    player.draw(screen)
    pygame.display.flip()  # Refresh display
    clock.tick(60)  # 60 FPS

pygame.quit()
